In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import OrderedDict

In [3]:
import os, sys
sys.path.append(os.getcwd())

In [4]:
from _element import feature_control as ft_c
from _element import varr
from _element import calculations as calc
from _element.result_control import Result

from _alg.prophet import Prophet_timeseries

from _evaluation.cross_validation import Cross_Validation
from _evaluation import compare

In [5]:
INPUT_FILENAME= 'KPP일별투입(10_17)_raw.xlsx'

In [6]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [7]:
y_col= df_raw['raw'].columns.values.tolist()

In [8]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [9]:
x_col= df_temp['raw'].columns.values.tolist()

In [10]:
df_raw['raw']['ds']= pd.to_datetime(df_raw['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_temp['raw']['ds']= pd.to_datetime(df_temp['raw']['ds'], box=True, format= '%Y/%m/%d', exact=True)
df_raw['raw']= pd.merge(df_raw['raw'], df_temp['raw'], how='inner', on='ds')

In [11]:
df_raw

{'raw':              ds       y   temp_max   temp_min  rain_amount
 0    2010-07-01   79590  30.500000  24.900000          0.0
 1    2010-07-02   79456  25.799999  22.700001         70.0
 2    2010-07-03   48469  25.600000  22.600000          1.0
 3    2010-07-04    1045  29.700001  23.100000          0.0
 4    2010-07-05   65049  30.600000  21.799999          0.0
 5    2010-07-06   84245  31.000000  22.400000          0.0
 6    2010-07-07   80493  29.600000  21.900000          0.0
 7    2010-07-08   73930  29.000000  21.299999          0.0
 8    2010-07-09   80166  29.299999  21.799999          0.0
 9    2010-07-10   49542  29.100000  22.900000          0.0
 10   2010-07-11    1411  25.900000  21.000000          0.0
 11   2010-07-12   67680  28.299999  22.200001          0.0
 12   2010-07-13   83587  31.299999  21.900000          0.0
 13   2010-07-14   73490  27.500000  22.700001          1.0
 14   2010-07-15   73332  29.799999  24.299999          0.0
 15   2010-07-16   74715  27.1000

In [57]:
r= Result()
initial_forecastdict= r.forecast_dict

In [13]:
prpt_basis= Prophet_timeseries(cv= True)

In [14]:
prpt_basis.add_model('basis', 'day', regressor= None)

In [15]:
cv= Cross_Validation(df_raw)

In [16]:
cv.slice('y', x_col)

In [17]:
cv._data

{'raw_0': {'test':              ds       y  temp_max  temp_min  rain_amount
  2696 2017-11-24  120647       2.9      -2.5          0.1
  2697 2017-11-25   67756       6.4       0.8         15.7
  2698 2017-11-26     100       6.4      -2.4          0.0
  2699 2017-11-27  117480       8.1       1.8          0.0
  2700 2017-11-28  122192       9.8       4.2          1.2
  2701 2017-11-29  120685       5.7      -3.2          0.0
  2702 2017-11-30  108386       1.8      -7.1          0.0,
  'testX':              ds  temp_max  temp_min  rain_amount
  2696 2017-11-24       2.9      -2.5          0.1
  2697 2017-11-25       6.4       0.8         15.7
  2698 2017-11-26       6.4      -2.4          0.0
  2699 2017-11-27       8.1       1.8          0.0
  2700 2017-11-28       9.8       4.2          1.2
  2701 2017-11-29       5.7      -3.2          0.0
  2702 2017-11-30       1.8      -7.1          0.0,
  'testY': 2696    120647
  2697     67756
  2698       100
  2699    117480
  2700    12219

In [18]:
prpt_basis.fit('basis', txs_traintest= cv._data)

In [19]:
prpt_basis.forecast('basis', forecast_dict= r.forecast_dict)

C:\Studying\Project_Nextop\nextop-engine\nextop_engine\_alg\prophet.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_forecast['ds'], box=True, format= '%Y/%m/%d', exact=True


In [20]:
r.err_rate()

In [21]:
r.avg_result('basis', delete_case= True)

{'forecast': [          ds       y  temp_max  temp_min  rain_amount           yhat
0 2017-11-24  120647       2.9      -2.5          0.1  120459.004517
1 2017-11-25   67756       6.4       0.8         15.7   76531.728142
2 2017-11-26     100       6.4      -2.4          0.0   20583.772102
3 2017-11-27  117480       8.1       1.8          0.0  106788.860212
4 2017-11-28  122192       9.8       4.2          1.2  118297.075805
5 2017-11-29  120685       5.7      -3.2          0.0  117146.076961
6 2017-11-30  108386       1.8      -7.1          0.0  115046.752933,           ds       y   temp_max   temp_min  rain_amount           yhat
0 2017-09-15  146743  27.299999  17.200001          0.0  127131.633881
1 2017-09-16   78827  26.100000  19.200001          0.0   82892.379655
2 2017-09-17     630  27.400000  17.500000          0.0   27038.015339
3 2017-09-18  124903  27.100000  19.200001          0.0  112254.209893
4 2017-09-19  142824  25.900000  14.900000          5.8  123613.654498
5 2017-

In [22]:
r.print_err_rate()

RMSE with segment basis_result: 15051.6768
MAPE with segment basis_result: 2692.5522
MAPE_with_std with segment basis_result: 27.9306
MAPE_div_std with segment basis_result: 112.7822
sMAPE with segment basis_result: 13.0802




In [23]:
r.forecast_dict.keys()

odict_keys(['basis_result'])

In [24]:
ft_c.add_division(df_raw, 'raw', 7, start_num= 3)

In [25]:
ft_c.divide_by(df_raw, 'raw', 'seasonality_7,3')

In [37]:
cv2= Cross_Validation(df_raw)

In [38]:
cv2.slice('y', x_col, forecastday= varr.FORECASTDAY//7)

In [39]:
cv2._data

{'(0,)_0': {'test':              ds       y   temp_max  temp_min  rain_amount  seasonality_7,3
  2657 2017-10-16  119379  20.200001       9.5          0.0                0
  2664 2017-10-23  121804  17.900000       9.8          0.0                0
  2671 2017-10-30  117899  12.300000       3.3          0.0                0
  2678 2017-11-06  113480  15.900000       9.8          0.0                0
  2685 2017-11-13  121625  11.000000       6.1          5.6                0
  2692 2017-11-20  119059   3.700000      -4.0          0.2                0
  2699 2017-11-27  117480   8.100000       1.8          0.0                0,
  'testX':              ds   temp_max  temp_min  rain_amount
  2657 2017-10-16  20.200001       9.5          0.0
  2664 2017-10-23  17.900000       9.8          0.0
  2671 2017-10-30  12.300000       3.3          0.0
  2678 2017-11-06  15.900000       9.8          0.0
  2685 2017-11-13  11.000000       6.1          5.6
  2692 2017-11-20   3.700000      -4.0      

In [58]:
prpt_basis.add_model('seasonality_7', 'week', regressor= ['temp_max', 'temp_min', 'rain_amount'])

In [59]:
prpt_basis.fit('seasonality_7', txs_traintest= cv2._data)

In [60]:
prpt_basis.forecast('seasonality_7', forecast_dict= r.forecast_dict)

C:\Studying\Project_Nextop\nextop-engine\nextop_engine\_alg\prophet.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_forecast['ds'], box=True, format= '%Y/%m/%d', exact=True


In [61]:
r.merge_result(['_0', '_1', '_2', '_3', '_4'])

{'forecast':           ds       y   temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-10-13  137316  18.000000       7.4          0.0                4   
0 2017-10-14   79129  19.700001       9.0          0.0                5   
0 2017-10-15      80  20.299999      14.1          0.0                6   
0 2017-10-16  119379  20.200001       9.5          0.0                0   
0 2017-10-17  132170  21.400000      14.8          0.0                1   
0 2017-10-18  133867  19.400000      12.3          0.5                2   
0 2017-10-19  131135  21.299999      11.5          0.0                3   
1 2017-10-20  139079  22.400000      11.3          0.0                4   
1 2017-10-21   74015  22.600000      10.8          0.0                5   
1 2017-10-22      19  21.500000       8.4          0.0                6   
1 2017-10-23  121804  17.900000       9.8          0.0                0   
1 2017-10-24  130253  18.600000       9.7          0.0                1   
1 2017-10-25

{'forecast':           ds       y   temp_max   temp_min  rain_amount  seasonality_7,3  \
0 2017-05-26  132142  21.299999  13.100000          0.0                4   
0 2017-05-27   67948  24.600000  15.100000          0.0                5   
0 2017-05-28     101  26.100000  18.299999          0.0                6   
0 2017-05-29  117597  27.900000  19.100000          0.0                0   
0 2017-05-30  130329  29.799999  17.700001          0.0                1   
0 2017-05-31  125349  28.200001  20.299999          0.0                2   
0 2017-06-01  124345  27.700001  17.100000          0.0                3   
1 2017-06-02  134160  24.799999  17.600000          0.2                4   
1 2017-06-03   71545  25.799999  15.800000          0.0                5   
1 2017-06-04     133  28.200001  16.900000          0.0                6   
1 2017-06-05  126649  29.100000  20.700001          0.0                0   
1 2017-06-06   73184  22.799999  14.400000         27.2                1   

{'forecast':           ds       y  temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-01-06  132151       9.5       1.2          0.0                4   
0 2017-01-07   70104       9.1       2.0          0.0                5   
0 2017-01-08      73      10.0       2.6          0.2                6   
0 2017-01-09  109699       4.0      -3.8          0.0                0   
0 2017-01-10  124362       0.9      -8.5          0.0                1   
0 2017-01-11  125760       0.4      -8.1          0.1                2   
0 2017-01-12  121618       0.0      -7.7          0.0                3   
1 2017-01-13  134991      -2.9     -10.6          0.4                4   
1 2017-01-14   79862      -5.9     -12.0          0.0                5   
1 2017-01-15      89      -2.4     -11.1          0.0                6   
1 2017-01-16  120018       0.9      -7.6          0.0                0   
1 2017-01-17  122902       2.2      -5.1          0.0                1   
1 2017-01-18  129416     

In [62]:
r.err_rate()
r.print_err_rate()

RMSE with segment _0_merged: 17094.1123
MAPE with segment _0_merged: 3956.0093
MAPE_with_std with segment _0_merged: 40.8916
MAPE_div_std with segment _0_merged: 87.2669
sMAPE with segment _0_merged: 14.4444


RMSE with segment _1_merged: 15527.6662
MAPE with segment _1_merged: 1270.5662
MAPE_with_std with segment _1_merged: 34.3139
MAPE_div_std with segment _1_merged: 75.7232
sMAPE with segment _1_merged: 11.4142


RMSE with segment _2_merged: 34504.4835
MAPE with segment _2_merged: 1057.4161
MAPE_with_std with segment _2_merged: 33.1267
MAPE_div_std with segment _2_merged: 68.7279
sMAPE with segment _2_merged: 23.8232


RMSE with segment _3_merged: 39146.5879
MAPE with segment _3_merged: 3897.8650
MAPE_with_std with segment _3_merged: 31.3488
MAPE_div_std with segment _3_merged: 74.8976
sMAPE with segment _3_merged: 30.4256


RMSE with segment _4_merged: 45726.6773
MAPE with segment _4_merged: 25908.6282
MAPE_with_std with segment _4_merged: 36.0161
MAPE_div_std with segment _4_merge

In [63]:
r.avg_result(str())

{'forecast': [          ds       y   temp_max  temp_min  rain_amount  seasonality_7,3  \
0 2017-10-13  137316  18.000000       7.4          0.0                4   
0 2017-10-14   79129  19.700001       9.0          0.0                5   
0 2017-10-15      80  20.299999      14.1          0.0                6   
0 2017-10-16  119379  20.200001       9.5          0.0                0   
0 2017-10-17  132170  21.400000      14.8          0.0                1   
0 2017-10-18  133867  19.400000      12.3          0.5                2   
0 2017-10-19  131135  21.299999      11.5          0.0                3   
1 2017-10-20  139079  22.400000      11.3          0.0                4   
1 2017-10-21   74015  22.600000      10.8          0.0                5   
1 2017-10-22      19  21.500000       8.4          0.0                6   
1 2017-10-23  121804  17.900000       9.8          0.0                0   
1 2017-10-24  130253  18.600000       9.7          0.0                1   
1 2017-10-2

In [64]:
r.print_err_rate()

RMSE with segment _result: 30399.9054
MAPE with segment _result: 7218.0969
MAPE_with_std with segment _result: 35.1394
MAPE_div_std with segment _result: 75.8707
sMAPE with segment _result: 23.1202




In [65]:
r.forecast_dict

OrderedDict([('_result',
              {'MAPE': 7218.0969474703625,
               'MAPE_div_std': 75.870703296154431,
               'MAPE_with_std': 35.139426491616348,
               'RMSE': 30399.905435440633,
               'forecast': [          ds       y   temp_max  temp_min  rain_amount  seasonality_7,3  \
                0 2017-10-13  137316  18.000000       7.4          0.0                4   
                0 2017-10-14   79129  19.700001       9.0          0.0                5   
                0 2017-10-15      80  20.299999      14.1          0.0                6   
                0 2017-10-16  119379  20.200001       9.5          0.0                0   
                0 2017-10-17  132170  21.400000      14.8          0.0                1   
                0 2017-10-18  133867  19.400000      12.3          0.5                2   
                0 2017-10-19  131135  21.299999      11.5          0.0                3   
                1 2017-10-20  139079  22.40000